In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale
import os

In [66]:
file = "epsilon-3"

### Load in `f` data

In [67]:
df = pd.read_csv(file + "-f.csv", sep=",")
df.T

,0,1,2,3,4,5,6,7,8,9,...,63937,63938,63939,63940,63941,63942,63943,63944,63945,63946
current_player,1.000000,0.000000,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,1.000000,0.0,...,1.000000,0.00000,1.000000,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,1.000000
is_current_player_hand_winner,0.000000,1.000000,0.000000,1.000000,0.000000,1.0,0.000000,1.000000,0.000000,1.0,...,1.000000,0.00000,1.000000,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,1.000000
is_current_player_game_winner,0.000000,1.000000,0.000000,1.000000,0.000000,1.0,0.000000,1.000000,0.000000,1.0,...,1.000000,0.00000,1.000000,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,1.000000
current_player_end_hand_score_advantage,-17.000000,17.000000,-17.000000,17.000000,-17.000000,17.0,-17.000000,17.000000,-17.000000,17.0,...,10.000000,-10.00000,10.000000,-10.000000,10.000000,-10.000000,10.00000,26.000000,-26.000000,26.000000
current_player_score,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,68.000000,0.00000,68.000000,0.000000,68.000000,0.000000,68.00000,78.000000,0.000000,78.000000
opponent_score,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,68.00000,0.000000,68.000000,0.000000,68.000000,0.00000,0.000000,78.000000,0.000000
current_player_deadwood,65.000000,44.000000,58.000000,44.000000,58.000000,43.0,58.000000,41.000000,55.000000,41.0,...,18.000000,26.00000,18.000000,26.000000,12.000000,18.000000,8.00000,25.000000,51.000000,10.000000
current_player_num_hit_cards,5.000000,3.000000,7.000000,3.000000,7.000000,5.0,7.000000,6.000000,7.000000,6.0,...,3.000000,5.00000,3.000000,5.000000,3.000000,2.000000,5.00000,8.000000,7.000000,2.000000
num_melds,0.000000,1.000000,0.000000,1.000000,0.000000,1.0,0.000000,1.000000,0.000000,1.0,...,2.000000,1.00000,2.000000,1.000000,2.000000,1.000000,2.00000,1.000000,1.000000,2.000000
point_sum_melds,0.000000,6.000000,0.000000,6.000000,0.000000,6.0,0.000000,6.000000,0.000000,6.0,...,56.000000,9.00000,56.000000,9.000000,56.000000,10.000000,56.00000,30.000000,29.000000,36.000000


In [68]:
f = [
#      'current_player',
#      'is_current_player_hand_winner',
#      'is_current_player_game_winner',
     'current_player_end_hand_score_advantage',
     'current_player_score',
     'opponent_score',
     'current_player_deadwood',
     'current_player_num_hit_cards',
     'num_melds',
     'point_sum_melds',
     'num_combos',
     'point_sum_combos',
     'num_knock_cache',
     'point_sum_knock_cache',
     'num_load_cards',
     'point_sum_load_cards',
     'turns_taken',
     'num_nearby_opponent_cards',
     'discard_danger'
]


x = f[1:]
y = f[0]

print("x =", x)
print("y =", y)

x = ['current_player_score', 'opponent_score', 'current_player_deadwood', 'current_player_num_hit_cards', 'num_melds', 'point_sum_melds', 'num_combos', 'point_sum_combos', 'num_knock_cache', 'point_sum_knock_cache', 'num_load_cards', 'point_sum_load_cards', 'turns_taken', 'num_nearby_opponent_cards', 'discard_danger']
y = current_player_end_hand_score_advantage


### Load in `m` data

In [69]:
card_matrices = []

with open(file + '-m.csv') as f:
    for line in f.readlines():
        card_matrices.append(eval(line))

card_matrices = np.array(card_matrices)
card_matrices

array([[[0.        , 0.26907994, 0.        , ..., 0.        ,
         0.26907994, 0.26907994],
        [0.26907994, 0.26907994, 0.26907994, ..., 0.26907994,
         0.26907994, 0.26907994],
        [0.26907994, 0.26907994, 0.26907994, ..., 0.26907994,
         0.26907994, 0.        ],
        ...,
        [0.26907994, 0.26907994, 0.26907994, ..., 0.26907994,
         0.26907994, 0.        ],
        [0.        , 0.26907994, 0.        , ..., 0.        ,
         0.26907994, 0.26907994],
        [0.26907994, 0.26907994, 0.26907994, ..., 0.26907994,
         0.26907994, 0.26907994]],

       [[0.26997829, 0.26986806, 0.27000489, ..., 0.        ,
         0.26110739, 0.        ],
        [0.26869792, 0.        , 0.26820064, ..., 0.24311868,
         0.29109001, 0.3146228 ],
        [0.26780297, 0.        , 0.        , ..., 0.21708868,
         0.25383154, 0.28658311],
        ...,
        [0.26780297, 0.        , 0.        , ..., 0.21708868,
         0.25383154, 0.28658311],
        [0.2

### split train and test data

In [70]:
trainFeaturesX, testFeaturesX, trainMatricesX, testMatricesX = train_test_split(df[x], card_matrices, test_size=0.25, random_state=42)
trainY, testY = train_test_split(df[[y]], test_size=0.25, random_state=42)

display(trainFeaturesX.T)
display(trainY.T)

,39308,30670,5046,35363,49543,7505,16796,3125,40012,30200,...,6265,54886,11284,44732,54343,62570,38158,860,15795,56422
current_player_score,16.000000,67.000000,0.000000,38.000000,0.000000,21.000000,21.000000,31.000000,40.0,72.000000,...,23.000000,59.00000,87.000000,34.000000,35.000000,0.00000,48.000000,36.000000,70.000000,27.00000
opponent_score,91.000000,54.000000,28.000000,19.000000,0.000000,24.000000,36.000000,34.000000,24.0,91.000000,...,20.000000,49.00000,78.000000,39.000000,58.000000,0.00000,94.000000,54.000000,91.000000,10.00000
current_player_deadwood,37.000000,42.000000,20.000000,28.000000,8.000000,6.000000,54.000000,28.000000,27.0,25.000000,...,14.000000,63.00000,36.000000,33.000000,10.000000,35.00000,22.000000,10.000000,38.000000,41.00000
current_player_num_hit_cards,6.000000,5.000000,3.000000,4.000000,3.000000,7.000000,7.000000,6.000000,4.0,7.000000,...,9.000000,4.00000,9.000000,5.000000,2.000000,6.00000,6.000000,4.000000,2.000000,9.00000
num_melds,1.000000,1.000000,2.000000,1.000000,2.000000,2.000000,0.000000,1.000000,1.0,1.000000,...,2.000000,0.00000,1.000000,1.000000,2.000000,1.00000,1.000000,2.000000,1.000000,1.00000
point_sum_melds,12.000000,27.000000,3.000000,16.000000,45.000000,36.000000,0.000000,9.000000,40.0,6.000000,...,45.000000,0.00000,21.000000,27.000000,67.000000,29.00000,18.000000,18.000000,12.000000,27.00000
num_combos,2.000000,2.000000,1.000000,2.000000,1.000000,3.000000,3.000000,3.000000,2.0,4.000000,...,3.000000,2.00000,4.000000,2.000000,0.000000,3.00000,2.000000,1.000000,1.000000,4.00000
point_sum_combos,25.000000,20.000000,8.000000,14.000000,2.000000,3.000000,26.000000,11.000000,14.0,13.000000,...,15.000000,18.00000,30.000000,20.000000,0.000000,24.00000,6.000000,0.000000,5.000000,37.00000
num_knock_cache,3.000000,3.000000,2.000000,3.000000,3.000000,4.000000,4.000000,4.000000,3.0,4.000000,...,3.000000,3.00000,3.000000,4.000000,3.000000,3.00000,4.000000,4.000000,4.000000,3.00000
point_sum_knock_cache,5.000000,4.000000,6.000000,5.000000,5.000000,2.000000,5.000000,5.000000,4.0,2.000000,...,7.000000,4.00000,4.000000,3.000000,7.000000,4.00000,3.000000,6.000000,6.000000,4.00000


,39308,30670,5046,35363,49543,7505,16796,3125,40012,30200,...,6265,54886,11284,44732,54343,62570,38158,860,15795,56422
current_player_end_hand_score_advantage,5.0,-15.0,5.0,-19.0,20.0,26.0,-34.0,-18.0,5.0,-4.0,...,-10.0,-19.0,10.0,6.0,16.0,-3.0,-13.0,36.0,39.0,55.0


### create the MLP model

In [72]:
mlp = Sequential()
mlp.add(Dense(10, input_dim=trainFeaturesX.shape[1], activation="relu"))
mlp.add(Dense(8, activation="relu"))
mlp.add(Dense(6, activation="relu"))
mlp.add(Dense(4, activation="relu"))
if False:
    mlp.add(Dense(1, activation = "linear"))

### Create CNN model

In [78]:
height = len(card_matrices[0])
width = len(card_matrices[0][0])
depth = 1
filters = (16, 32, 64)

inputShape = (height, width, depth)
chanDim = -1

inputs = Input(shape=inputShape)

for (i, f) in enumerate(filters):
    # if this is the first CONV layer then set the input appropriately
    if i == 0:
        x = inputs

    # CONV => RELU => BN => POOL
    x = Conv2D(f, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

# flatten the volume, then FC => RELU => BN => DROPOUT
x = Flatten()(x)
x = Dense(16)(x)
x = Activation("relu")(x)
x = BatchNormalization(axis=chanDim)(x)
x = Dropout(0.5)(x)

# match the number of nodes coming out of the MLP
x = Dense(4)(x)
x = Activation("relu")(x)

if False:
    x = Dense(1, activation="linear")(x)

cnn = Model(inputs, x)

### Combine

In [79]:
combinedInput = concatenate([mlp.output, cnn.output])

x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

model = Model(inputs=[mlp.input, cnn.input], outputs=x)

### Compile

In [80]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

### Train

In [82]:
model.fit(
    [trainFeaturesX, trainMatricesX], trainY,
    validation_data=([testFeaturesX, testMatricesX], testY),
    epochs=400, batch_size=8)

Epoch 1/400
5995/5995 [==============================] - 29s 5ms/step - loss: 98.8659 - val_loss: 98.4372
Epoch 2/400
5995/5995 [==============================] - 30s 5ms/step - loss: 98.0657 - val_loss: 97.5914
Epoch 3/400
5995/5995 [==============================] - 28s 5ms/step - loss: 97.4788 - val_loss: 97.3891
Epoch 4/400
5995/5995 [==============================] - 28s 5ms/step - loss: 97.0618 - val_loss: 96.7939
Epoch 5/400
5995/5995 [==============================] - 29s 5ms/step - loss: 97.0372 - val_loss: 97.7761
Epoch 6/400
5995/5995 [==============================] - 27s 5ms/step - loss: 96.6855 - val_loss: 96.6412
Epoch 7/400
5995/5995 [==============================] - 29s 5ms/step - loss: 96.3643 - val_loss: 96.6307
Epoch 8/400
5995/5995 [==============================] - 29s 5ms/step - loss: 96.2870 - val_loss: 96.1323
Epoch 9/400
5995/5995 [==============================] - 27s 5ms/step - loss: 95.8507 - val_loss: 95.8049
Epoch 10/400
5995/5995 [======================

### Testing

In [1]:
preds = model.predict([testFeaturesX, testMatricesX])
diff = (preds.T[0] - testY.values.T[0])**2

print("average difference =", np.average(diff))

NameError: name 'model' is not defined

In [84]:
print(model.predict([testFeaturesX, testMatricesX]).T[0][:10])

[ 1.0398519  1.901139   7.8905153 -5.9691067  3.540642  -0.9991611
 -5.1615887 -5.578822  -0.9995842  1.0667367]


In [85]:
print(testY.values.T[0][:10])

[ 37. -18.  24. -11.   3. -12.   5. -13. -28.   4.]


In [86]:
model.save('epsilon.h5')